In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

from sklearn import linear_model
from sklearn.metrics  import mean_squared_error as mse

from bokeh.plotting import figure, show, output_notebook
from bokeh.charts import Scatter, show
output_notebook()

# with open('dataframe.pkl', 'rb') as f:
#     df = pkl.load(f)
    
models = ['regr', 'regr_ey', 'regr_log', 'nb1NN', 'nb2NN', 'nb3NN']

Loading BokehJS ...

In [ ]:
# uploading data from files
path = '../messy_data/'
with open(path+'df.pkl', 'rb') as f:
    df_regr = pkl.load(f)
with open(path+'dataframeALL_regr_ey.pkl', 'rb') as f:
    df_ey = pkl.load(f)
with open(path+'dataframeALL_regr_log.pkl', 'rb') as f:
    df_log = pkl.load(f)

In [2]:
# uploading data from folder pkl_data
path = '../pkl_data/'

with open(path+'avg_data.pkl', 'rb') as f:
	avg_data = pkl.load(f)

# df_err_*_diff dataframe with differences with and without a model *
with open(path+'df_err_regr.pkl', 'rb') as f:
    df_err_regr = pkl.load(f)
with open(path+'df_err_regr_ey.pkl', 'rb') as f:
    df_err_regr_ey = pkl.load(f)
# with open(path+'df_err_regr_log.pkl', 'rb') as f:
#     df_err_regr_logx = pkl.load(f)
with open(path+'df_err_nb1NN.pkl', 'rb') as f:
    df_err_nb1NN = pkl.load(f)
with open(path+'df_err_nb2NN.pkl', 'rb') as f:
    df_err_nb2NN = pkl.load(f)
with open(path+'df_err_nb3NN.pkl', 'rb') as f:
    df_err_nb3NN = pkl.load(f)
with open(path+'df_err_nomodel.pkl', 'rb') as f:
    df_err_nomodel = pkl.load(f)

with open(path+'ind_models.pkl', 'rb') as f:
    ind_models = pkl.load(f)

In [ ]:
def plot_coef(df, title, name):
# fun for plotting dataframe created by script fit_model.py
# but it's not working for k-NN models
# df.loc[0]['model B'] wskazuje na modele wszystkich agentów, którzy byli w interakcję z agentem 0, 
# więc wystarczy dodać df.loc[0,1]['model A'] aby mieć modele wszystkich agentów
    X = [x.intercept_ for x in df.loc[0][name+' model B'].values]
    Y = [y.coef_[0] for y in df.loc[0][name+' model B'].values]
    
    X = X+[x.intercept_ for x in df.loc[0,1][name+' model A'].values]
    Y = Y+[y.coef_[0] for y in df.loc[0,1][name+' model A'].values]
    print(len(X))
    
    f = figure(title=title)
    f.xaxis.axis_label = 'Intercept'
    f.yaxis.axis_label = 'Coefficient'
    f.scatter(x=X, y=Y, color='navy', size=6, alpha=.2)
    show(f)
    print('intercept standard deviation: ', np.std(X))
    print('coefficient standard deviation: ', np.std(Y))
    
def mse_count(df, model):
# function for counting mean squared error for (d_A^2 + d_B^2)/2
    X = [x for x in df[model+' d_A'].values]
    Y = [y for y in df[model+' d_B'].values]
    mse = 0
    for (x,y) in zip(X,Y):
        mse += (x*x + y*y)/2
    return mse/len(X)

def mse_count_dA(df, model):
# function for counting mean squared error for d_A^2
    X = [x for x in df[model+' d_A'].values]
    mse = 0
    for x in X:
        mse += x*x
    return mse/len(X)

def ind_mse_plot(df, model):
#fun for plotting individual mse errors
    n = 31
    X = [x*x for x in df[model+' d_A'].values]
    Y = [y*y for y in df[model+' d_B'].values]
    f = figure(title="Individual MSE for "+model)
    for i in range(int(len(X)/n)):
        ind_X = X[i*n:i*(n+1)]
        ind_Y = Y[i*n:i*(n+1)]
        mse = 0
        for (x,y) in zip(ind_X, ind_Y):
            mse += (x*x + y*y)/2
        f.scatter(x=mse/n, color='navy', size=6, alpha=.3)
    show(f)


In [ ]:
# intercept / coef plots
plot_coef(df_regr, 'intercept / coef '+"regr", "regr")

In [ ]:
# intercept / coef plots
plot_coef(df_ey, 'intercept / coef '+"ey", 'regr_ey')

In [ ]:
# intercept / coef plots
plot_coef(df_log, 'intercept / coef log', 'regr_log')

In [ ]:
# MSE (d_A^2+d_B^2)/2
for (df, model) in [(df_regr, 'regr'), (df_ey, 'regr_ey'), (df_log, 'regr_log')]:
    print('MSE '+model+': ',mse_count(df, model))

In [ ]:
# MSE d_A^2
for (df, model) in [(df_regr, 'regr'), (df_ey, 'regr_ey'), (df_log, 'regr_log')]:
    print('MSE '+model+': ',mse_count_dA(df, model))

Wyniki uzyskane w lutym (średnie d_A^2)
MSE regr:     1178.05986567
MSE regr_ey:  3200.93215805
MSE regr_log: 473686394475.0
MSE nb1NN:    2111.55143449
MSE nb2NN:    1618.70770597
MSE nb3NN:    1432.190942
MSE zero:     1970.91485889

In [ ]:
# czy tutaj nie powinno być przypadkiem to samo, co w komórce wyżej?
temp = []
for i, lin_mod in enumerate(df_regr.loc[0]['regr model B'].values):
    temp.append(lin_mod.predict(avg_data['mean'][i%31])[0])
print('MSE: regr: ', mse(temp, list(avg_data['mean'])*243))

temp = []
for i, lin_mod in enumerate(df_ey.loc[0]['regr_ey model B'].values):
    temp.append(lin_mod.predict(avg_data['mean'][i%31])[0])
print('MSE regr ey: ', mse(temp, list(avg_data['mean'])*243))

temp = []
for i, lin_mod in enumerate(df_log.loc[0]['regr_log model B'].values):
    temp.append(lin_mod.predict(avg_data['mean'][i%31])[0])
print('MSE regr log: ', mse(temp, list(avg_data['mean'])*243))


In [ ]:
# Średnie MSE dopasowania poszczególnych modeli dla każdego uczestnika (też z walidacją leave-one-out).

ind_mse_regr, ind_mse_regr_ey, ind_mse_regr_log = [], [], []
# regr
temp = []
for i, lin_mod in enumerate(df_regr.loc[0]['regr model B'].values):
    temp.append(lin_mod.predict(avg_data['mean'][i%31])[0])
    if (i%31 == 30):
        ind_mse_regr.append(mse(temp, avg_data['mean']))
        temp = []
# regr_ey
temp = []
for i, lin_mod in enumerate(df_ey.loc[0]['regr_ey model B'].values):
    temp.append(lin_mod.predict(avg_data['mean'][i%31])[0])
    if (i%31 == 30):
        ind_mse_regr_ey.append(mse(temp, avg_data['mean']))
        temp = []
# regr_log
temp = []
for i, lin_mod in enumerate(df_log.loc[0]['regr_log model B'].values):
    temp.append(lin_mod.predict(avg_data['mean'][i%31])[0])
    if (i%31 == 30):
        ind_mse_regr_log.append(mse(temp, avg_data['mean']))
        temp = []

print('mean ind MSE regr: ',np.mean(ind_mse_regr))
print('mean ind MSE regr_ey: ', np.mean(ind_mse_regr_ey))
print('mean ind MSE regr_log: ', np.mean(ind_mse_regr_log))

print("ind MSE regr standard deviation: ", np.std(ind_mse_regr))
print("ind MSE regr_ey standard deviation: ", np.std(ind_mse_regr_ey))
print("ind MSE regr_log standard deviation: ", np.std(ind_mse_regr_log))

In [ ]:
# TODO liczy się w nieskończoność
ind_mse_plot(df_regr, 'regr')